<a href="https://colab.research.google.com/github/agmCorp/colab/blob/main/GoldenSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np

# 1. Preprocesamiento de imágenes
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),   # Redimensionar a tamaño que espera ResNet
    transforms.ToTensor(),           # Convertir a tensor
    transforms.Normalize(            # Normalizar con los valores de ImageNet
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def load_image(path):
    img = Image.open(path).convert("RGB")
    return preprocess(img).unsqueeze(0)  # Añadir dimensión batch

# 2. Cargar ResNet50 preentrenada en ImageNet
model = models.resnet50(pretrained=True)
# Quitar la capa final de clasificación (nos quedamos con los embeddings de 2048 dims)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

# 3. Función para extraer embedding
def get_embedding(img_tensor):
    with torch.no_grad():
        embedding = model(img_tensor)
    return embedding.squeeze().numpy()  # Devolver como vector 1D

# 4. Calcular similitud coseno
def cosine_similarity(vec1, vec2):
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot / (norm1 * norm2)




/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 162MB/s]


In [5]:
# 5. Ejemplo de uso
img1 = load_image("/content/sample_data/samplePicture.png")  # Caja de referencia (Golden)
img2 = load_image("/content/wrong2.png")    # Caja real tomada en línea

emb1 = get_embedding(img1)
emb2 = get_embedding(img2)

similarity = cosine_similarity(emb1, emb2)
print(f"Similitud coseno: {similarity:.4f}")

# Ejemplo de decisión simple
umbral = 0.95
if similarity >= umbral:
    print("✅ Caja correcta (similar a Golden Sample)")
else:
    print("❌ Posible error visual en el packaging")

Similitud coseno: 0.9728
✅ Caja correcta (similar a Golden Sample)
